# Predicting NFL Win Totals

## BASELINE

## constraints
- years: 2009-present
- all 32 teams


## Model
- Random Forest

## Dependent Variable
- Current Year's Win Total

## Independent Variables
- [x]  Previous Year's Strength of Schedule
- [ ] Current Year's Market-Priced Strength of Schedule
- Previous Year's Relative Performance (EPA Standardized w.r.t Year)

In [1]:
# import python packages & libraries
import numpy as np
import pandas as pd
import nfl_data_py as nfl

In [2]:
# YEARS = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]
#debug
YEARS = [2019, 2020]

# Get Data
- Previous Year's Strength of Schedule
    - each opposing team's previous year's Win PCT
        - Obtain schedule for the `result` for `game_type` REG season (ref: data_dict https://github.com/nflverse/nflreadr/blob/main/data-raw/dictionary_schedules.csv)
        - Transform schedule result to win/loss
        - Group Year & Team to obtain sum of wins / count games played
- Current Year's Market-Priced Strength of Schedule
    - obtain team wins total odds
        - ref: https://www.sportsoddshistory.com/nfl-win/?y=2022&sa=nfl&t=win&o=t
    - calc opposing team's expected Win PCT
- Previous Year's Relative Performance
    - Team's EPA standardized w.r.t Year's mean team EPA
    

## Current Year's Market-Priced Strength of Schedule

In [3]:
# Create for-loop to grab sportsoddshistory pages. 
#  add season YEAR to the DataFrame
#  concat the data to create a single DataFrame
#  adjust team_name

# import nfl schedule and 
# calculate opposing team's market-line of win pct

In [4]:
nfl_market_exp_wins = pd.DataFrame()

for year in YEARS:
    sportsoddshistory = pd.read_html(f'https://www.sportsoddshistory.com/nfl-win/?y={year}&sa=nfl&t=win&o=t')
    sportsoddshistory[0]['season_n'] = year
    nfl_market_exp_wins = pd.concat([nfl_market_exp_wins,sportsoddshistory[0]], ignore_index=True)

In [5]:
nfl_market_exp_wins.rename(columns={'Team':'full_name'},inplace=True)

In [6]:
full_team_names = nfl_market_exp_wins['full_name'].unique()

In [7]:
full_team_names

array(['Arizona Cardinals', 'Atlanta Falcons', 'Baltimore Ravens',
       'Buffalo Bills', 'Carolina Panthers', 'Chicago Bears',
       'Cincinnati Bengals', 'Cleveland Browns', 'Dallas Cowboys',
       'Denver Broncos', 'Detroit Lions', 'Green Bay Packers',
       'Houston Texans', 'Indianapolis Colts', 'Jacksonville Jaguars',
       'Kansas City Chiefs', 'Los Angeles Chargers', 'Los Angeles Rams',
       'Miami Dolphins', 'Minnesota Vikings', 'New England Patriots',
       'New Orleans Saints', 'New York Giants', 'New York Jets',
       'Oakland Raiders', 'Philadelphia Eagles', 'Pittsburgh Steelers',
       'San Francisco 49ers', 'Seattle Seahawks', 'Tampa Bay Buccaneers',
       'Tennessee Titans', 'Washington Redskins', 'Las Vegas Raiders',
       'Washington Football Team'], dtype=object)

In [8]:

full_team_names = ['Arizona Cardinals',
                   'Atlanta Falcons',
                   'Baltimore Ravens',
                   'Buffalo Bills',
                   'Carolina Panthers',
                   'Chicago Bears',
                   'Cincinnati Bengals',
                   'Cleveland Browns',
                   'Dallas Cowboys',
                   'Denver Broncos',
                   'Detroit Lions',
                   'Green Bay Packers',
                   'Houston Texans',
                   'Indianapolis Colts',
                   'Jacksonville Jaguars',
                   'Kansas City Chiefs',
                   'Las Vegas Raiders',
                   'Los Angeles Chargers',
                   'Los Angeles Rams',
                   'Miami Dolphins',
                   'Minnesota Vikings',
                   'New England Patriots',
                   'New Orleans Saints',
                   'New York Giants',
                   'New York Jets',
                   'Oakland Raiders',
                   'Philadelphia Eagles',
                   'Pittsburgh Steelers',
                   'San Diego Chargers',
                   'San Francisco 49ers',
                   'Seattle Seahawks',
                   'St Louis Rams',
                   'Tampa Bay Buccaneers',
                   'Tennessee Titans',
                   'Washington Football Team',
                   'Washington Redskins']
team_abbr = ['ARI',
             'ATL',
             'BAL',       
             'BUF',
             'CAR',
             'CHI',
             'CIN',
             'CLE',
             'DAL',
             'DEN',
             'DET',
             'GB',
             'HOU',
             'IND',
             'JAX',
             'KC',
             'LV',
             'LAC',
             'LA',
             'MIA',
             'MIN',
             'NE',
             'NO',
             'NYG',
             'NYJ',
             'OAK',
             'PHI',
             'PIT',
             'SD',
             'SF',
             'SEA',
             'STL',
             'TB', 
             'TEN',
             'WAS',
             'WAS'
             
]

# print(len(team_abbr) )
# print(len(full_team_names))
teams_df = pd.DataFrame({'full_name':full_team_names,'team_name':team_abbr})


In [9]:
nfl_market_exp_wins = pd.merge(nfl_market_exp_wins, teams_df, how='left')

In [10]:
nfl_market_exp_wins['full_name'].unique()

array(['Arizona Cardinals', 'Atlanta Falcons', 'Baltimore Ravens',
       'Buffalo Bills', 'Carolina Panthers', 'Chicago Bears',
       'Cincinnati Bengals', 'Cleveland Browns', 'Dallas Cowboys',
       'Denver Broncos', 'Detroit Lions', 'Green Bay Packers',
       'Houston Texans', 'Indianapolis Colts', 'Jacksonville Jaguars',
       'Kansas City Chiefs', 'Los Angeles Chargers', 'Los Angeles Rams',
       'Miami Dolphins', 'Minnesota Vikings', 'New England Patriots',
       'New Orleans Saints', 'New York Giants', 'New York Jets',
       'Oakland Raiders', 'Philadelphia Eagles', 'Pittsburgh Steelers',
       'San Francisco 49ers', 'Seattle Seahawks', 'Tampa Bay Buccaneers',
       'Tennessee Titans', 'Washington Redskins', 'Las Vegas Raiders',
       'Washington Football Team'], dtype=object)

In [12]:
assert len(nfl_market_exp_wins[nfl_market_exp_wins['team_name'].isna()]) == 0

In [13]:
nfl_market_exp_wins

,full_name,Win Total,Over Odds,Under Odds,Week betsettled,ActualWins,Result,season_n,team_name
0,Arizona Cardinals,5.0,-120,100,Week 17,5,Push,2019,ARI
1,Atlanta Falcons,8.5,-140,120,Week 12,7,Under,2019,ATL
2,Baltimore Ravens,8.5,110,-130,Week 12,14,Over,2019,BAL
3,Buffalo Bills,7.0,-140,120,Week 12,10,Over,2019,BUF
4,Carolina Panthers,8.0,-130,110,Week 15,5,Under,2019,CAR
...,...,...,...,...,...,...,...,...,...
59,San Francisco 49ers,10.5,120,-140,Week 10,6,Under,2020,SF
60,Seattle Seahawks,9.5,100,-120,Week 15,12,Over,2020,SEA
61,Tampa Bay Buccaneers,9.5,-125,105,Week 16,11,Over,2020,TB
62,Tennessee Titans,8.5,-130,110,Week 14,11,Over,2020,TEN
